# Choosing the Right Metric

This module introduced both the K Nearest Neighbors model as well as a variety of different metrics for classification.  It is important to select and understand the appropriate metric for your task.  This exercise is meant to get practice considering the difference between these new classification metrics and accompanying evaluation tools. Specifically, explore datasets related to business from the UCI Machine Learning Repository [here](https://archive.ics.uci.edu/datasets).  

Select a dataset of interest and clearly state the classification task.  Specifically, describe a business problem that could be solved using the dataset and a KNN classification model.  Further, identify what you believe to be the appropriate metric and justify your choice.  Build a basic model with the `KNearestNeighbor` and grid search to optimize towards your chosen metric.  Share your results with your peers.

# Imports

In [1]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np

np.random.seed(1234)

import pandas as pd

pd.set_option("display.max_columns", None)

import matplotlib.pyplot as plt

plt.rcParams.update({"axes.grid": True, "figure.figsize": (16, 4)})

import plotly.express as px

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV

import seaborn as sns

sns.set()

from scipy.io import arff

# Data Load

Rice data set, from UCI's machine learning data repository, [here](https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik).

In [2]:
arff_contents = arff.loadarff("./data/Rice_Cammeo_Osmancik.arff")
df = pd.DataFrame(arff_contents[0])
df

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231.0,525.578979,229.749878,85.093788,0.928882,15617.0,0.572896,b'Cammeo'
1,14656.0,494.311005,206.020065,91.730972,0.895405,15072.0,0.615436,b'Cammeo'
2,14634.0,501.122009,214.106781,87.768288,0.912118,14954.0,0.693259,b'Cammeo'
3,13176.0,458.342987,193.337387,87.448395,0.891861,13368.0,0.640669,b'Cammeo'
4,14688.0,507.166992,211.743378,89.312454,0.906691,15262.0,0.646024,b'Cammeo'
...,...,...,...,...,...,...,...,...
3805,11441.0,415.858002,170.486771,85.756592,0.864280,11628.0,0.681012,b'Osmancik'
3806,11625.0,421.390015,167.714798,89.462570,0.845850,11904.0,0.694279,b'Osmancik'
3807,12437.0,442.498993,183.572922,86.801979,0.881144,12645.0,0.626739,b'Osmancik'
3808,9882.0,392.296997,161.193985,78.210480,0.874406,10097.0,0.659064,b'Osmancik'


In [3]:
df["Class"] = df["Class"].str.decode("ascii")

In [4]:
df

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231.0,525.578979,229.749878,85.093788,0.928882,15617.0,0.572896,Cammeo
1,14656.0,494.311005,206.020065,91.730972,0.895405,15072.0,0.615436,Cammeo
2,14634.0,501.122009,214.106781,87.768288,0.912118,14954.0,0.693259,Cammeo
3,13176.0,458.342987,193.337387,87.448395,0.891861,13368.0,0.640669,Cammeo
4,14688.0,507.166992,211.743378,89.312454,0.906691,15262.0,0.646024,Cammeo
...,...,...,...,...,...,...,...,...
3805,11441.0,415.858002,170.486771,85.756592,0.864280,11628.0,0.681012,Osmancik
3806,11625.0,421.390015,167.714798,89.462570,0.845850,11904.0,0.694279,Osmancik
3807,12437.0,442.498993,183.572922,86.801979,0.881144,12645.0,0.626739,Osmancik
3808,9882.0,392.296997,161.193985,78.210480,0.874406,10097.0,0.659064,Osmancik


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   float64
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   float64
 6   Extent             3810 non-null   float64
 7   Class              3810 non-null   object 
dtypes: float64(7), object(1)
memory usage: 238.2+ KB


# Data Set Up

## Shuffle the Data

Because it is all Cammeo, then all Osmancik

In [6]:
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
df

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,12442.0,459.535004,187.508850,87.187302,0.885323,12941.0,0.587580,Cammeo
1,12408.0,437.014008,179.741165,88.829605,0.869343,12598.0,0.636928,Osmancik
2,12867.0,449.079987,181.700562,91.341064,0.864460,13152.0,0.649062,Osmancik
3,13090.0,472.945007,202.601578,83.230179,0.911722,13331.0,0.775290,Cammeo
4,10359.0,409.510986,173.337967,76.875809,0.896273,10510.0,0.573588,Osmancik
...,...,...,...,...,...,...,...,...
3805,16625.0,535.989014,229.793594,93.089622,0.914272,16951.0,0.654141,Cammeo
3806,13901.0,478.848999,200.441910,89.341988,0.895170,14232.0,0.568548,Cammeo
3807,16291.0,523.192993,223.252335,93.604156,0.907859,16595.0,0.581157,Cammeo
3808,10847.0,417.924011,170.366791,82.473007,0.875018,11107.0,0.746319,Osmancik


## Split into X=features and y=target

In [7]:
X, y = df.drop(columns=["Class"]), df["Class"]
y

0         Cammeo
1       Osmancik
2       Osmancik
3         Cammeo
4       Osmancik
          ...   
3805      Cammeo
3806      Cammeo
3807      Cammeo
3808    Osmancik
3809    Osmancik
Name: Class, Length: 3810, dtype: object

Also map the target. Don't have to do this, but it makes the confusion matrices a bit easier to work with

In [8]:
target_map = {"Cammeo": 0, "Osmancik": 1}
y = y.map(target_map)
y

0       0
1       1
2       1
3       0
4       1
       ..
3805    0
3806    0
3807    0
3808    1
3809    1
Name: Class, Length: 3810, dtype: int64

## Split into Training and Dev

Stratify to keep balance of classes same between all data and the splits

In [9]:
X_train, X_dev, y_train, y_dev = train_test_split(
    X, y, random_state=42, stratify=y
)

In [10]:
y.value_counts(normalize=True)

Class
1    0.572178
0    0.427822
Name: proportion, dtype: float64

In [11]:
y_train.value_counts(normalize=True)

Class
1    0.572279
0    0.427721
Name: proportion, dtype: float64

In [12]:
y_dev.value_counts(normalize=True)

Class
1    0.571878
0    0.428122
Name: proportion, dtype: float64

# Model Definition

Pipeline of
- Scaling on the features - all alreay numeric
- KNeighborsClassifier classifier


In [13]:
knn_pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier()),
    ]
)

knn_pipe

Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

# Grid Search

What's the best metric here? Not sure yet!

In [14]:
param_grid={"knn__n_neighbors": list(range(1, 51, 5))}
param_grid

{'knn__n_neighbors': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46]}

In [15]:
knn_grid = GridSearchCV(
    estimator=knn_pipe,
    param_grid=param_grid,
    scoring="recall",
    cv=5,
).fit(X, y)

In [16]:
knn_grid.best_params_

{'knn__n_neighbors': 41}

In [17]:
best_knn_pipe = knn_grid.best_estimator_
best_knn_pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=41))])